In [194]:
import pandas as pd

In [195]:
# nutri_df = pd.read_csv('tmp/nutri_list.csv')
nutri_list_df = pd.read_csv('data/nutri_list_raw_edited.csv')

In [196]:
nutri_list_df

,nutri_name
0,cystine
1,vitamin b12
2,ash
3,phosphorus
4,lactoferrin
...,...
177,taurin
178,epa
179,otpp
180,soluble dietary fiber


In [197]:
nutri_df = nutri_list_df

In [198]:
import re
def preprocess_term(term):
    term = term.replace('total', ' ')
    term = term.strip()

    term = term.replace('of which', ' ')
    term = term.strip()

    term = term.replace(',', ';')
    term = term.replace('(', ';').replace(')', ';').lower()
    term = term.strip(";").strip()
    term = re.sub(r'vitamin (\w\w?\w?)', 'vitamin \\1 ; \\1 ;', term)
    term = term.strip(";").strip()
    return term

In [199]:
nutri_df['preprocessed_term'] = nutri_df['nutri_name'].apply(preprocess_term)

In [200]:
nutri_df

,nutri_name,preprocessed_term
0,cystine,cystine
1,vitamin b12,vitamin b12 ; b12
2,ash,ash
3,phosphorus,phosphorus
4,lactoferrin,lactoferrin
...,...,...
177,taurin,taurin
178,epa,epa
179,otpp,otpp
180,soluble dietary fiber,soluble dietary fiber


In [201]:
term_list = nutri_df['preprocessed_term'].tolist()

In [202]:
term_list = [i for i in term_list if i != '']
term_list = list(set(term_list))

In [203]:
len(term_list)

172

In [204]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers

In [205]:
password='FtxkVZYOjZBdsLqbryvc'
ca_certs_dir='/home/vinhvan/elasticsearch-8.4.2/config/certs/http_ca.crt'
es = Elasticsearch(
            "https://localhost:9200",
            ca_certs=ca_certs_dir,
            basic_auth=("elastic", password),
            request_timeout=60
        )
print(es.info())

{'name': 'VINHVAN-ITC-AI', 'cluster_name': 'elasticsearch', 'cluster_uuid': 'nGpADS-xR_2jfOXdEXGYXw', 'version': {'number': '8.4.2', 'build_flavor': 'default', 'build_type': 'tar', 'build_hash': '89f8c6d8429db93b816403ee75e5c270b43a940a', 'build_date': '2022-09-14T16:26:04.382547801Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


In [239]:
index_name = 'nutri_parse_1'

es.options(ignore_status=[400,404]).indices.delete(index=index_name)

# create mapping
setting = {
            "similarity": {
                "scripted_tfidf": {
                    "type": "scripted",
                    "script": {
                        # "source": "double tf = Math.sqrt(doc.freq); double idf = Math.log((field.docCount+1.0)/(term.docFreq+1.0)) + 1.0; double norm = 1/Math.sqrt(doc.length); return query.boost * tf * idf * norm;"
                        # this tf idf score remove idf
                        "source": "double tf = Math.sqrt(doc.freq); double idf =  1.0; double norm = 1/Math.sqrt(doc.length + 1); return query.boost * tf * idf * norm;"
                    }
                },
                "custom_bm25": {
                    "type": "BM25",
                    "b": 0.65,
                    "k1": 1.2
                }

            },
            "analysis": {
                "analyzer": {
                    "my_standard_shingle_no_unique": {
                        "tokenizer": "standard",
                        "filter": ["lowercase", "my_shingle_filter"]
                    }
                },
                "filter": {
                    "my_shingle_filter": {
                        "type": "shingle",
                        "min_shingle_size": 2,
                        "max_shingle_size": 2,
                        "output_unigrams": True
                    }
                }
            }
        }

mapping = {
    "properties": {
        "full_name_no_unique": {
            "type": "text",
            "index_phrases": True,
            "store": True,
            "analyzer": "my_standard_shingle_no_unique",
            # "similarity": "custom_bm25"
        },
        "presentative_name": {
            "type": "keyword",
            "index": False,
            "store": True
        }
    }
}

es.indices.create(index=index_name, mappings=mapping, settings=setting)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'nutri_parse_1'})

In [240]:
sep_string = ' cq3rq3asd '
def preprocess_for_search(term):
    toks = term.split(';')
    toks = [i.strip() for i in toks if i.strip() != '']
    # sort toks by length desc
    toks = sorted(toks, key=lambda x: len(x), reverse=True)
    term = sep_string.join(toks)
    term = sep_string + term + sep_string
    return term


In [241]:
data = [preprocess_for_search(term) for term in term_list]

In [242]:
data

[' cq3rq3asd crom cq3rq3asd ',
 ' cq3rq3asd vitamin b1 cq3rq3asd thiamin cq3rq3asd b1 cq3rq3asd ',
 ' cq3rq3asd nh3 cq3rq3asd ',
 ' cq3rq3asd caffeine cq3rq3asd ',
 ' cq3rq3asd molybdenum cq3rq3asd mo cq3rq3asd ',
 ' cq3rq3asd ara cq3rq3asd ',
 ' cq3rq3asd l carnitine cq3rq3asd ',
 ' cq3rq3asd lutein cq3rq3asd ',
 ' cq3rq3asd protid cq3rq3asd ',
 ' cq3rq3asd added sugar cq3rq3asd ',
 ' cq3rq3asd dietary fiber cq3rq3asd ',
 ' cq3rq3asd vitamin b3 cq3rq3asd b3 cq3rq3asd ',
 ' cq3rq3asd monounsaturated fat cq3rq3asd ',
 ' cq3rq3asd unsaturated fat cq3rq3asd ',
 ' cq3rq3asd beta caroten cq3rq3asd ',
 ' cq3rq3asd vitamin pp cq3rq3asd pp cq3rq3asd ',
 ' cq3rq3asd no2 cq3rq3asd ',
 ' cq3rq3asd calcium cq3rq3asd ca cq3rq3asd ',
 ' cq3rq3asd phospho cq3rq3asd ',
 ' cq3rq3asd cholin cq3rq3asd ',
 ' cq3rq3asd nucleotid cq3rq3asd ',
 ' cq3rq3asd riboflavin cq3rq3asd vitamin b2 cq3rq3asd b2 cq3rq3asd ',
 ' cq3rq3asd pufa cq3rq3asd ',
 ' cq3rq3asd soluble dietary fiber cq3rq3asd ',
 ' cq3rq3asd salt

In [243]:
presentative_name_list = nutri_df['nutri_name'].tolist()

In [244]:
[term for term in' cq3rq3asd acid linoleic cq3rq3asd '.split(sep_string) if term != ''][0]

'acid linoleic'

In [245]:
# index into es db
import time

# assert len(data) == len(presentative_name_list)

for k in range(0, len(data), 1000):
    # print(i)
    # print(len(raw_dict_terms))
    # print(data[i:i+1000])
    actions = [
        {
            "_index": index_name,
            "_source": {
                "full_name_no_unique": i,
                "presentative_name": [term for term in i.split(sep_string) if term != ''][0].strip()
            }
        }
        for i in data[k:k+1000]
    ]
    helpers.bulk(es, actions)

print("DATA SIZE: ", len(data))
time.sleep(2)
resp = es.count(index=index_name, query={"match_all": {}})
print(resp) 

DATA SIZE:  172
{'count': 172, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}}


In [246]:
def get_word(text):
    return re.findall(string=text, pattern=r'[\w\d]+', flags=re.MULTILINE | re.IGNORECASE | re.UNICODE)


def preprocess_query(text):
    text = text.lower()
    text = " ".join(get_word(text))
    text = sep_string + text + sep_string
    return text


def get_match_term_query(raw_term, rank_list_size=10):
    j = {
        # "explain": True,
        "query": {
            "bool": {
                "should": [
                    {"match": {"full_name_no_unique": {"query": raw_term, "boost": 1}}},
                ],
                "minimum_should_match": 1
            }
        },
        "_source": {"includes": ["full_name_no_unique", "presentative_name"]},
        'from': 0,
        'size': rank_list_size
    }

    return j


def check_term_contain(term, text):
    def add_sep(toks):
        sep_string = "|||"
        return sep_string + sep_string.join(toks) + sep_string
    
    # respect word order
    # text_toks = add_sep(get_word(text))
    # term_toks = add_sep(get_word(term))
    # if term_toks in text_toks:
    #     return True
    # return False

    # disregard word order
    text_toks = set(get_word(text))
    term_toks = set(get_word(term))
    if term_toks.issubset(text_toks):
        return True
    return False


def search_term(text):
    text = preprocess_query(text)
    # print("preprocessed query: ", text)
    query = get_match_term_query(text)
    resp = es.search(index=index_name, body=query)
    ret = list()
    for  h in resp['hits']['hits']:
        if h['_score'] < 1.8:
            continue

        terms = h['_source']['full_name_no_unique'].replace(sep_string, ';').strip().strip(';').strip().split(';')
        keep = False
        found_term = None
        for term in terms:
            term = term.strip()
            if check_term_contain(term, text):
                found_term = term
                keep = True
                break

        if not keep:
            continue

        d = {
            'term': found_term,
            'presentative_name': h['_source']['presentative_name']
        }
        ret.append(d)
    return ret

In [247]:
check_term_contain('5 kcal', 'energy,55 kcal,sodium,0.01 g,carbohydrate,14 g,total sugars,14 g,vitamin C,41 mg')

False

In [248]:
search_term('energy,55 kcal,sodium,0.01 g,carbohydrate,14 g,total sugars,14 g,vitamin C,41 mg')

/tmp/ipykernel_481105/2542155770.py:55: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  resp = es.search(index=index_name, body=query)


[{'term': 'vitamin c', 'presentative_name': 'vitamin c'},
 {'term': 'carbohydrate', 'presentative_name': 'carbohydrate'},
 {'term': 'sugars', 'presentative_name': 'sugars'},
 {'term': 'energy', 'presentative_name': 'energy'},
 {'term': 'sodium', 'presentative_name': 'sodium'}]

In [249]:
# TODO: tokenize then search to match each token, take into account word boundary

import re
def seperate_phrase(text):
    subst = " quantitiytoken "
    # regex = r"(?:(\<\=|\>\=|\<|\>) *)?((?:\b\d+ *(?:[\.\,] *\d+)? *(?:\-|\~|\÷|\±) *)?\b\d+ *(?:[\.\,] *\d+)?) *(\%|µg|iu|i\.u|kcal|kj|ug|meq|mcg|mg|kg|g|ml|fl|lít|l|µg|cfu)"
    
    # capture BB-12,1x 10^9 cfu
    # (?:(\<\=|\>\=|\<|\>) *)?(?:(?:\b\d+(?:[\.\,]\d+)? ?[x|\*] ?10\^-?\d+)|(?:\b\d+(?:[\.\,]\d+)? ?(?:\-|\~) ?)?\b\d+(?:[\.\,]\d+)?) ?(\%|µg|iu|kcal|kj|ug|meq|mcg|mg|kg|g|ml|fl|lít|l|µg|cfu)
    regex = r"(?:(\<\=|\>\=|\<|\>) *)?((?:\b\d+(?:[\.\,]\d+)? *[x|\*|×] *10\^-?\d+)|(?:\b\d+ *(?:[\.\,] *\d+)? *(?:\-|\~|\÷|\±) *)?\b\d+ *(?:[\.\,] *\d+)?) *(\%|µg|iu|i\.u|kcal|kj|ug|meq|mcg|mg|kg|g|ml|fl|lít|l|µg|cfu)"
    result = re.sub(regex, subst, text, 0, re.MULTILINE | re.UNICODE | re.IGNORECASE)

    list_quant = re.findall(regex, text, re.MULTILINE | re.UNICODE | re.IGNORECASE)
    ret = result.split(subst)
    # ret = [phrase.strip().strip(',').strip() for phrase in ret]
    ret = [" ".join(re.findall(string=phrase, pattern=r"[\w\d]+", 
                               flags=re.MULTILINE | re.UNICODE | re.IGNORECASE)) for phrase in ret]
    ret = [i for i in ret if i != '']

    return ret, list_quant

In [250]:
test_string = """energy,465 kcal,protein,17 g,tryptophan,220 mg,total fat,20 g,acid linoleic,3800 mg,acid alpha-linoleic,300 mg,ara,11 mg,dha,83.5 mg,carbohydrate,53 g,dietary fiber,3 g,2'-FL HMO,0.18 g,taurine,40 mg,nucleotid,20 mg,lutein,136 µg,moisture,3 g,sodium,250 mg,potassium,830 mg,cl,470 mg,calcium,780 mg,phosphorus,580 mg,magnesium,60 mg,manganese,350 µg,iron,8.5 mg,iot,110 µg,zinc,5.4 mg,Cu,0.35 mg,selen,21 µg,vitamin A,1790 iu,vitamin d3,416 iu,vitamin E,10.5 mg TE,vitamin k1,46 µg,vitamin C,90 mg,vitamin B1,0.75 mg,vitamin B2,1.4 mg,vitamin pp,7 mg,vitamin B6,1 mg,follic acid,180 µg,pantothenic acid,3.7 mg,vitamin B12,2.9 µg,biotin,22 µg,cholline,156 mg,BB-12,1 x 10^9 cfu,LGG,1 x 10^9 cfu"""

In [251]:
phrases, quant = seperate_phrase(text=test_string)

In [253]:
# for phrase in phrases:
#     print(phrase)
#     print(search_term(phrase))
#     print()

In [257]:
def parse_nutri(text):
    phrases, quants = seperate_phrase(text=text)
    ret = list()
    for phrase, quant in zip(phrases, quants):
        ret.append({
            'phrase': phrase,
            'quant': quant,
            'match': search_term(phrase)
        })
    return ret

In [258]:
parse_nutri(test_string)

/tmp/ipykernel_481105/2542155770.py:55: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  resp = es.search(index=index_name, body=query)


[{'phrase': 'energy',
  'quant': ('', '465 ', 'kcal'),
  'match': [{'term': 'energy', 'presentative_name': 'energy'}]},
 {'phrase': 'protein',
  'quant': ('', '17 ', 'g'),
  'match': [{'term': 'protein', 'presentative_name': 'protein'}]},
 {'phrase': 'tryptophan',
  'quant': ('', '220 ', 'mg'),
  'match': [{'term': 'tryptophan', 'presentative_name': 'tryptophan'}]},
 {'phrase': 'total fat',
  'quant': ('', '20 ', 'g'),
  'match': [{'term': 'fat', 'presentative_name': 'fat'}]},
 {'phrase': 'acid linoleic',
  'quant': ('', '3800 ', 'mg'),
  'match': [{'term': 'acid linoleic', 'presentative_name': 'acid linoleic'}]},
 {'phrase': 'acid alpha linoleic',
  'quant': ('', '300 ', 'mg'),
  'match': [{'term': 'acid alpha linoleic',
    'presentative_name': 'acid alpha linoleic'},
   {'term': 'acid linoleic', 'presentative_name': 'acid linoleic'}]},
 {'phrase': 'ara',
  'quant': ('', '11 ', 'mg'),
  'match': [{'term': 'ara', 'presentative_name': 'ara'}]},
 {'phrase': 'dha',
  'quant': ('', '83.5'

In [295]:
def pretty_print_nutri(parsed_nutri_list):
    ret = list()
    for nutri in parsed_nutri_list:
        # print(nutri['match'][0]['presentative_name'])
        if len(nutri['match']) == 0:
            presentative = nutri['phrase'] + "(UNKNOWN)"
        else:
            presentative = nutri['match'][0]['term']
        s = presentative + ': ' + " ".join([i.strip() for i in nutri['quant']]).strip()
        ret.append(s.strip())
    return "; ".join(ret)

In [296]:
pretty_print_nutri(parse_nutri(test_string))

/tmp/ipykernel_481105/2542155770.py:55: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  resp = es.search(index=index_name, body=query)


'energy: 465 kcal; protein: 17 g; tryptophan: 220 mg; fat: 20 g; acid linoleic: 3800 mg; acid alpha linoleic: 300 mg; ara: 11 mg; dha: 83.5 mg; carbohydrate: 53 g; dietary fiber: 3 g; 2 fl hmo: 0.18 g; taurine: 40 mg; nucleotid: 20 mg; lutein: 136 µg; moisture: 3 g; sodium: 250 mg; potassium: 830 mg; cl: 470 mg; calcium: 780 mg; phosphorus: 580 mg; magnesium: 60 mg; manganese: 350 µg; iron: 8.5 mg; iot: 110 µg; zinc: 5.4 mg; cu: 0.35 mg; selen: 21 µg; vitamin a: 1790 iu; vitamin d3: 416 iu; vitamin e: 10.5 mg; vitamin k1: 46 µg; vitamin c: 90 mg; vitamin b1: 0.75 mg; vitamin b2: 1.4 mg; vitamin pp: 7 mg; vitamin b6: 1 mg; follic acid: 180 µg; pantothenic acid: 3.7 mg; vitamin b12: 2.9 µg; biotin: 22 µg; cholline: 156 mg; bb: 12,1 x 10^9 cfu; LGG(UNKNOWN): 1 x 10^9 cfu'

In [297]:
raw_df = pd.read_csv('data/Bitewise DB - Master product list _ Ingredients & Nutrition facts - Master list.csv')

In [298]:
def pd_parse_nutri(x):
    # check nan or empty
    if pd.isna(x) or x == '':
        return ''
    return pretty_print_nutri(parse_nutri(x))

In [299]:
raw_df['parsed_nutri'] = raw_df['Nutrition facts'].apply(pd_parse_nutri)

/tmp/ipykernel_481105/2542155770.py:55: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  resp = es.search(index=index_name, body=query)


In [301]:
raw_df.to_csv('data/parsed_nutri.csv', index=False)